# UDFinstaller #
This notebook demonstrates how to use `pyTigerGraph` for installing UDF to be able to install those algorithmms that requires these UDF and do preproccessing tasks on graphs stored in `TigerGraph`.

## Connection to Database ##
The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    useCert=False
)

In [5]:
conn.gsql('LS')

'---- Graph Cora\nVertex Types:\n- VERTEX Paper(PRIMARY_ID id INT, x LIST<INT>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL, fastrp_embedding LIST<DOUBLE>) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"\nEdge Types:\n- DIRECTED EDGE Cite(FROM Paper, TO Paper, attr BOOL)\n\nGraphs:\n- Graph Cora(Paper:v, Cite:e)\nJobs:\nQueries:\n- test(string attr) (installed v2)\n- test2(string attr) (installed v2)\n- tg_fastRP(set<string> v_type, set<string> e_type, string weights, float beta, int k, int reduced_dim, int sampling_constant, int random_seed, bool print_accum, string result_attr)\n- tg_pagerank(string v_type, string e_type, float max_change, int max_iter, float damping, int top_k, bool print_accum, string result_attr, string file_path, bool display_edges) (installed v2)\n- user_defined_query() (installed v2)\n\n\n\n'

# Install UDF
Install the desired UDF by providing the address where it is located.

In [3]:
conn.installUDF("/Users/neda.zarayeneh/pyTigerGraph/tests/ExprFunctions.hpp")
conn.installUDF("/Users/neda.zarayeneh/pyTigerGraph/tests/ExprUtil.hpp")

ExprFunctions installed succesfully !
ExprFunctions installed succesfully !


## Get fastRP as a feature ##
The fastRP algorithm is available in GDS library called tg_fastRP under the class of community detection algorithms  https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/GraphML/Embeddings/FastRP/tg_fastRP.gsql. fastRP needs UDF to be installed before being able to install and run it. In the previous cell, the required UDF gets installed and now fastRP can be used to creat feature.

In [9]:
f = conn.gds.featurizer()
f.installAlgorithm(query_name="tg_fastRP",global_change=True)

'tg_fastRP'

In [16]:
params = {'v_type': 'Paper', 'e_type': ['Cite','reverse_Cite'], 'weights': '1,1,2', 'beta': -0.85, 'k': 3, 'reduced_dim': 128, 
          'sampling_constant': 1, 'random_seed': 42, 'print_accum': False,'result_attr':"",'file_path' :""}
f.runAlgorithm(query_name='tg_fastRP',params=params,feat_name ="fastrp_embedding",global_schema=True,timeout = 2147480)

[]


# Install User Defined Query

In [28]:
query = '''
CREATE QUERY user_defined_query3() FOR GRAPH Cora { 
  /* Write query logic here */ 
  PRINT "user_defined_query works!"; 
}'''

print(query)


CREATE QUERY user_defined_query3() FOR GRAPH Cora { 
  /* Write query logic here */ 
  PRINT "user_defined_query works!"; 
}


In [29]:
outFileName="./user_defined_query1.gsql"
outFile=open(outFileName, "w")
outFile.write(query)
outFile.close()

In [30]:
f.installAlgorithm(query,"./user_defined_query1.gsql" )

Installing and optimizing the queries, it might take a minute


'CREATE QUERY user_defined_query3() FOR GRAPH Cora { \n  /* Write query logic here */ \n  PRINT "user_defined_query works!"; \n}'